<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Address-queries" data-toc-modified-id="Address-queries-1">Address queries</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Process" data-toc-modified-id="Process-1.0.1">Process</a></span></li><li><span><a href="#Calculating-distance" data-toc-modified-id="Calculating-distance-1.0.2">Calculating distance</a></span><ul class="toc-item"><li><span><a href="#Sorting" data-toc-modified-id="Sorting-1.0.2.1">Sorting</a></span></li><li><span><a href="#Finding-50-closest-neighbors-for-those-with-new-polling-places" data-toc-modified-id="Finding-50-closest-neighbors-for-those-with-new-polling-places-1.0.2.2">Finding 50 closest neighbors for those with new polling places</a></span></li></ul></li></ul></li><li><span><a href="#Finding-control-sets" data-toc-modified-id="Finding-control-sets-1.1">Finding control sets</a></span></li></ul></li></ul></div>

# Address queries

### Process
 - First, need to sort by lat/long, since this is only an O(N log N) operation 
 - Then, for each of the ones _with a new polling place_, which reduces it 10-fold, find the 100 closest locations (50 up and 50 down unless it's at the top or bottom)
 - Calculate distance between each polling and its closest 100 locations  
 - Instead of 2,000,000 x 2,000,000 calculations with the 0.1 degrees approach, this is only 200,000 x 100 

In [31]:
from dask import dataframe as dd
from pygeocoder import Geocoder as geo
from math import radians, cos, sin, asin, sqrt
import numpy as np
import pandas as pd

In [32]:
# Loading data and dropping duplicates
df = pd.read_csv('NC_combined_files_geo.tsv', sep='\t')

In [33]:
df.head()

,Unnamed: 0,ncid,county_desc,voter_status_desc,voter_status_reason_desc,race_code,address,poll_changed,election_desc,voting_method,...,vtd_label,election_year,voted,precinct,county_name,polling_place_name,poll_address,coords,latitude,longitude
0,0,CY22868,MONTGOMERY,ACTIVE,VERIFIED,W,"334 W ALLENTON ST ,MT GILEAD,NC 27306",0,11/08/2016 GENERAL,CURBSIDE,...,MTG,2016.0,1.0,MT GILEAD,MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,"106 ALLENTON ST, MT GILEAD, NC 27306","(35.2134165, -80.0066842)",35.213417,-80.006684
1,1,CY22873,MONTGOMERY,ACTIVE,VERIFIED,B,"103 RANCE LN ,MT GILEAD,NC 27306",0,11/08/2016 GENERAL,IN-PERSON,...,MTG,2016.0,1.0,MT GILEAD,MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,"106 ALLENTON ST, MT GILEAD, NC 27306","(35.2030978, -80.0104126)",35.203098,-80.010413
2,2,CY22900,MONTGOMERY,ACTIVE,VERIFIED,B,"587 PARKERTOWN RD ,MT GILEAD,NC 27306",0,NaN,NaN,...,NaN,2016.0,0.0,MT GILEAD,MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,"106 ALLENTON ST, MT GILEAD, NC 27306","(35.2283188, -80.012684)",35.228319,-80.012684
3,3,CY22935,MONTGOMERY,ACTIVE,VERIFIED,A,"4188 NC HWY 73 ,MT GILEAD,NC 27306",0,NaN,NaN,...,NaN,2016.0,0.0,MT GILEAD,MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,"106 ALLENTON ST, MT GILEAD, NC 27306","(35.1872377, -79.98682219999999)",35.187238,-79.986822
4,4,CY22949,MONTGOMERY,ACTIVE,VERIFIED,B,"5035 NC HWY 109 ,MT GILEAD,NC 27306",0,11/08/2016 GENERAL,PROVISIONAL,...,MTG,2016.0,1.0,MT GILEAD,MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,"106 ALLENTON ST, MT GILEAD, NC 27306","(35.1518339, -80.01172249999999)",35.151834,-80.011722


In [34]:
def lat_long_columns ():
    
    '''
    Function that creates latitude and longitude columns from the coords column
    '''

    latitude_list = []
    longitude_list = []
    column_list = df['coords'].tolist()

    for count, elem in enumerate(df['coords'].tolist()):
        latitutde = float(column_list[count].split(',')[0].replace('(', ''))
        latitude_list.append(latitutde)

        longitude = float(column_list[count].split(',')[1].replace(')', '').replace(' ', ''))
        longitude_list.append(longitude)
    
    df['latitude'] = latitude_list
    df['longitude'] = longitude_list
    
    return df

In [35]:
# Running the lat/long column function
df = lat_long_columns()

In [37]:
# Adding column for if they voted and removing some unnecessary columns
df['voted'] = np.where(df['voting_method'].isna(), 0, 1)


df = df[['ncid', 'race_code', 'voter_status_desc', 'address', 'county_desc', 'polling_place_name', 'voting_method',
        'voted', 'precinct', 'poll_address', 'poll_changed', 'latitude', 'longitude']]

df.head()

,ncid,race_code,voter_status_desc,address,county_desc,polling_place_name,voting_method,voted,precinct,poll_address,poll_changed,latitude,longitude
0,CY22868,W,ACTIVE,"334 W ALLENTON ST ,MT GILEAD,NC 27306",MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,CURBSIDE,1,MT GILEAD,"106 ALLENTON ST, MT GILEAD, NC 27306",0,35.213417,-80.006684
1,CY22873,B,ACTIVE,"103 RANCE LN ,MT GILEAD,NC 27306",MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,IN-PERSON,1,MT GILEAD,"106 ALLENTON ST, MT GILEAD, NC 27306",0,35.203098,-80.010413
2,CY22900,B,ACTIVE,"587 PARKERTOWN RD ,MT GILEAD,NC 27306",MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,NaN,0,MT GILEAD,"106 ALLENTON ST, MT GILEAD, NC 27306",0,35.228319,-80.012684
3,CY22935,A,ACTIVE,"4188 NC HWY 73 ,MT GILEAD,NC 27306",MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,NaN,0,MT GILEAD,"106 ALLENTON ST, MT GILEAD, NC 27306",0,35.187238,-79.986822
4,CY22949,B,ACTIVE,"5035 NC HWY 109 ,MT GILEAD,NC 27306",MONTGOMERY,MOUNT GILEAD FIRE DEPARTMENT,PROVISIONAL,1,MT GILEAD,"106 ALLENTON ST, MT GILEAD, NC 27306",0,35.151834,-80.011722


### Calculating distance

Apparently this approximation method works well for small distances (e.g., less than 500 miles) and is very quick (100,000 calculations per second)  
https://stackoverflow.com/questions/15736995/how-can-i-quickly-estimate-the-distance-between-two-latitude-longitude-points

^^^ Tried a few after looking it up in Google Maps and it's directionally correct. Need to strip out the negatives for long tho, or change function. Worth doing a few more to confirm 

In [38]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    miles = km*0.62
    return miles

#### Sorting

In [39]:
# Sorting by lat and long
df = df.sort_values(by=['latitude', 'longitude'])
df.head()

,ncid,race_code,voter_status_desc,address,county_desc,polling_place_name,voting_method,voted,precinct,poll_address,poll_changed,latitude,longitude
1755,CY26194,W,ACTIVE,"506 S EAST BEACH DR ,BALD HEAD ISLAND,NC 28461",BRUNSWICK,THE BRUNSWICK CENTER AT SOUTHP,ABSENTEE ONESTOP,1,SOUTHPORT 1,"1513 HOWE ST, SOUTHPORT, NC 28461",0,33.852689,-77.961172
3877,BY374394,W,ACTIVE,"615 SUNSET LAKES BLVD,SUNSET BEACH,NC 28468",BRUNSWICK,SEA TRAIL'S PROPERTY OWNERS AS,NaN,0,SHINGLETREE 1,"200 STATION TRL, SUNSET BEACH, NC 28468",0,33.890140,-78.528587
3452,BY373275,W,ACTIVE,"1807 SHALLOTTE DR ,HIGH POINT,NC 27262",GUILFORD,WESLEY MEMORIAL UMC,IN-PERSON,1,H02,"1225 CHESTNUT DR, HIGH POINT, NC 27262",0,33.981293,-78.380100
3273,EL2749,W,ACTIVE,"821 GLENARTHUR DR ,WILMINGTON,NC 28412",NEW HANOVER,IMMACULATE CONCEPTION CATHOLIC,ABSENTEE ONESTOP,1,FP07,"6650 CAROLINA BEACH RD, WILMINGTON, NC 28412",0,34.095443,-77.899939
1744,CY25809,W,ACTIVE,"1102 NAVAHO TRL ,WILMINGTON,NC 28409",NEW HANOVER,PARSLEY ELEMENTARY SCHOOL,ABSENTEE ONESTOP,1,M02,"3530 MASONBORO LOOP RD, WILMINGTON, NC 28409",0,34.166784,-77.873641


#### Finding 50 closest neighbors for those with new polling places

In [40]:
# Resetting index (will need to discuss how to do this part in dask) then getting indices w/ new locations
df = df.reset_index()
new_poll_indeces = df.index[df['poll_changed'] == 1].tolist()

In [41]:
new_poll_indeces

[262,
 694,
 701,
 704,
 707,
 708,
 731,
 810,
 811,
 868,
 869,
 1041,
 1271,
 1492,
 1493,
 1546,
 1741,
 1742,
 1744,
 1748,
 1749,
 1750,
 1751,
 1753,
 1757,
 1758,
 1759,
 1760,
 1766,
 1779,
 1780,
 1781,
 1783,
 1785,
 1786,
 1795,
 1796,
 1797,
 1798,
 1799,
 1800,
 1801,
 1802,
 1803,
 1806,
 1816,
 1819,
 1820,
 1823,
 1825,
 1833,
 1835,
 1836,
 1837,
 1838,
 1841,
 1844,
 1845,
 1846,
 1851,
 1854,
 1856,
 1857,
 1858,
 1859,
 1860,
 1861,
 1863,
 1864,
 1868,
 1869,
 1870,
 1874,
 1878,
 1885,
 1891,
 1898,
 1899,
 1901,
 1904,
 1905,
 1912,
 1916,
 1919,
 1920,
 1925,
 1926,
 1928,
 1931,
 1936,
 1938,
 1942,
 1946,
 1948,
 1950,
 1951,
 1952,
 1953,
 1954,
 1960,
 1984,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1998,
 1999,
 2000,
 2001,
 2002,
 2010,
 2015,
 2017,
 2018,
 2019,
 2020,
 2025,
 2028,
 2031,
 2032,
 2034,
 2035,
 2036,
 2037,
 2038,
 2039,
 2040,
 2041,
 2043,
 2044,
 2045,
 2046,
 2050,
 2052,
 2053,
 2055,
 2056,
 2057,
 2058,
 2061,
 20

In [42]:
# Creating dataframe with NCID and indices
def ncid_to_index ():
    '''
    Creates a dataframe that maps NCID to that individual's index. Used in later cleaning.
    '''
    ncid_list = []
    for count, elem in enumerate (new_poll_indeces):
        ncid_list.append(df['ncid'][count])

    return pd.DataFrame({'ncid': ncid_list, 'index': new_poll_indeces})

In [43]:
ncid_to_index_df = ncid_to_index()

In [44]:
ncid_to_index_df.head()

,ncid,index
0,CY26194,262
1,BY374394,694
2,BY373275,701
3,EL2749,704
4,CY25809,707


In [45]:
# New subset of the dataframe to be used later 

index_voted_df = df['voted']

In [46]:
def fifty_nearest (new_poll_indeces):
    '''
    Params:
        new_poll_indices: list of indices where that voter's polling location changed 
    
    Returns:
        A dictionary where each key is the index of a voter whose and the each
        value is a list of indices of his 50 closest neighbors and whether that person voted
    
    Note:
        This only returns indices of neighbors who did not have their location changed, 
        and does not return the individual himself. 
    '''
    
    fifty_nearest_dict = {}

    for elem in new_poll_indeces:
        unfiltered_list = np.linspace(elem - 34, elem + 35, 70)
        removed_movers_list = [elem for elem in unfiltered_list if elem not in new_poll_indeces]
        fifty_nearest_dict[elem] = removed_movers_list[11:61]
    
    return fifty_nearest_dict

In [47]:
# Creating the dict
fifty_nearest_dict = fifty_nearest(new_poll_indeces)

In [48]:
def generate_column_names ():
    '''
    Simple function to generate column names for the 50 closest neighbors to be used in distance matrix
    '''
    col_names = []
    for elem_one in range(50):
        for elem_two in range(2):
            if elem_two == 0:
                col_names.append("Neighbor" + str(elem_one+1) + "Distance")
            else:
                col_names.append("Neighbor" + str(elem_one+1) + "Voted")
    
    return col_names

In [49]:
# Having to manually update for one since it's at the end 
fifty_nearest_dict[5088] = [elem for elem in range(5063, 5103)]
fifty_nearest_dict[5091] = [elem for elem in range(5063, 5103)]
fifty_nearest_dict[5098] = [elem for elem in range(5063, 5103)]

In [50]:
def generate_rows ():
    '''
    This function creates an array of rows (each of which is another array) to eventually
    be passed into the dataframe 
    '''
    
    list_of_rows = []
    for key in fifty_nearest_dict.keys():

        row_values = []
        for count, elem in enumerate(fifty_nearest_dict[key]):
            row_values.append(haversine(df['latitude'][key], df['longitude'][key], df['latitude'][elem], df['longitude'][elem]))
            current_index = fifty_nearest_dict[key][count] 
            row_values.append(index_voted_df[current_index])
        list_of_rows.append(row_values)
        row_values = []
    
    return list_of_rows  

??? Is there a way to parallelize this? This is the bottleneck right now 

In [51]:
# Generating the rows for the dataframe
rows_for_df = generate_rows()

# Creating the dataframe and adding a column for the initial index
neighbor_df = pd.DataFrame(rows_for_df, columns = generate_column_names())
neighbor_df['initial_index'] = new_poll_indeces

In [52]:
# Merging with other df to get ncid column 
final_df = pd.merge(ncid_to_index_df, neighbor_df, left_on='index', right_on='initial_index', how='inner').drop(['initial_index'], axis=1)

In [53]:
# Adding column for whether the treated individual voted, then reordering columns 
final_df['voted'] = [index_voted_df[elem] for elem in final_df['index']] 

cols = list(final_df)
reordered_cols = cols.insert(0, cols.pop(cols.index('voted')))
final_df = final_df.loc[:, cols]

cols = list(final_df)
reordered_cols = cols.insert(0, cols.pop(cols.index('ncid')))
final_df = final_df.loc[:, cols]
final_df = final_df.drop(['index'], axis=1)

In [54]:
final_df.head()

,ncid,voted,Neighbor1Distance,Neighbor1Voted,Neighbor2Distance,Neighbor2Voted,Neighbor3Distance,Neighbor3Voted,Neighbor4Distance,Neighbor4Voted,...,Neighbor46Distance,Neighbor46Voted,Neighbor47Distance,Neighbor47Voted,Neighbor48Distance,Neighbor48Voted,Neighbor49Distance,Neighbor49Voted,Neighbor50Distance,Neighbor50Voted
0,CY26194,1,2.359397,1,1.216454,1,1.216454,1,0.602677,1,...,3.756680,0.0,18.374854,1.0,12.354832,1.0,12.354832,1.0,12.354832,1.0
1,BY374394,1,2.298578,1,15.791276,1,15.128370,1,0.352594,1,...,65.890297,1.0,15.433507,1.0,15.351437,1.0,14.807952,1.0,2.561293,1.0
2,BY373275,1,1.178164,1,15.123272,1,15.609959,1,2.702237,1,...,150.999125,1.0,14.968810,1.0,1.884839,1.0,10.201571,0.0,14.736500,1.0
3,EL2749,0,2.660844,1,14.929349,1,15.059044,1,15.059044,1,...,10.244925,0.0,14.780048,1.0,14.780048,1.0,14.819928,1.0,14.819928,1.0
4,CY25809,1,15.194801,1,1.985632,0,0.221970,1,15.857365,1,...,14.954795,1.0,14.954795,1.0,15.410766,1.0,14.997547,1.0,14.997547,1.0


This shows that it seemed to work as expected. On average, the middle neighbors should be the closest and the outside neighbors should be farther away:     

In [55]:
print (final_df['Neighbor2Distance'].mean())
print (final_df['Neighbor24Distance'].mean())
print (final_df['Neighbor25Distance'].mean())
print (final_df['Neighbor48Distance'].mean())

39.24701333188198
39.98338600405339
47.05820799879956
55.26325649030604


Would also be good to plot average distance from 1-50 and make sure it goes down towards the middle

## Finding control sets

For each row (person whose poll was changed), filter it to be people within X miles and then pick the X closest.    